In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 取各仓库主题标签的第一个为代表
topic_csv = pd.read_csv('final_topics.csv')
topic_list = topic_csv['topics']
topic_csv_len = len(topic_csv)
main_topic_csv = pd.read_csv('main256_topic.csv')
main_topic_list = main_topic_csv['topics']
represent_topic_list = []
max_len = 0 
topic_len_list = dict()
for i in range(topic_csv_len):
    topic_len = len(eval(topic_list[i]))
    if max_len < topic_len:
        max_len = topic_len
    if topic_len not in topic_len_list.keys():
        topic_len_list.update({topic_len:1})
    else:
        topic_len_list[topic_len] +=1
    represent_topic = eval(topic_list[i])[0]
    count256 = 0
    for j in range(256):
        if represent_topic == main_topic_list[j]:
            represent_topic_list.append(j)
            break
        count256 += 1
        if count256 ==256:
            print(represent_topic)
print(represent_topic_list)
print('各仓库最多标签数：',max_len)
print(topic_len_list)

In [ ]:
# 提取代表标签的位置并保存（空白对照组）
print(len(represent_topic_list))
def create_identity_matrix(n):
    identity_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        identity_matrix[i][i] = 1
    return identity_matrix
# 调用函数生成 256*256 的单位矩阵
identity_feature = create_identity_matrix(256)
print(identity_feature)
represent_feature = []
for i in represent_topic_list:
    represent_feature.append(identity_feature[i])
txt_filename = 'feature_vector/feature_without.txt' ##
with open(txt_filename,'w',encoding='utf-8') as f:
    f.write(str(represent_feature))
    f.write('\n')   
print(f"Data saved to {txt_filename} successfully.")

In [ ]:
# 提取代表标签的主题关联度特征并保存
represent_feature = []
txt_filename = 'topic_matrix/topic_matrix_collaborate.txt' ##
with open(txt_filename,'r',encoding='utf-8') as f:
    feature_matrix = eval(f.read())
    for i in represent_topic_list:
        represent_feature.append(feature_matrix[i])
txt_filename = 'feature_vector/feature_collaborate.txt' ##
with open(txt_filename,'w',encoding='utf-8') as f:
    f.write(str(represent_feature))
    f.write('\n')   
print(f"Data saved to {txt_filename} successfully.")

In [ ]:
# Multi-hot 编码处理仓库多标签
topic_csv = pd.read_csv('final_topics.csv')
topic_list = topic_csv['topics']
topic_csv_len = len(topic_csv)
main_topic_csv = pd.read_csv('main256_topic.csv')
main_topic_list = main_topic_csv['topics']
y = []
for i in range(topic_csv_len):
    one_y = [0]*256
    one_topic_list = eval(topic_list[i])
    one_len = len(one_topic_list)
    for item in one_topic_list:
        for j in range(256):
            if item == main_topic_list[j]:
                one_y[j] = 1
                break
    y.append(one_y)
print(len(y),len(y[3]))
txt_filename = 'y.txt'
with open(txt_filename,'w',encoding='utf-8') as f:
    f.write(str(y))
print(f"Labels saved to {txt_filename} successfully.")

In [ ]:
# 先单独处理 readme 文件（转至 r_d_processing.ipynb 文件详细处理）
readme_list = pd.read_csv('final_infors.csv')
readme = readme_list['readme']
readme.to_csv('observe/readme_origin.csv',index=0)

In [ ]:
# 翻译 \x 编码
from urllib import parse
readme_csv = pd.read_csv('observe/readme_link4.csv')
readme_list = readme_csv['readme']
readme_csv_len = len(readme_list)
print("仓库大小为：",readme_csv_len)
readme_k = [] # 
for i in range(readme_csv_len):
    one_readme = readme_list[i][2:-1]
    #print(one_readme)
    one_readme_len = len(one_readme)
    #print(one_readme_len)
    s =  one_readme.encode('unicode_escape')
    #print(s)
    ss = s.decode('utf-8').replace('\\\\x', '%')
    #print(ss)
    new_readme = parse.unquote(ss)
    #print(new_readme)
    readme_k.append(new_readme) #
    #print(len(new_readme)) 
print(len(readme_k)) #
csv_filename = 'observe/readme_chinese.csv' #
with open(csv_filename, 'a', encoding='utf-8') as f:
    for i in range(len(readme_k)): #
        f.write("\"")
        f.write(readme_k[i]) #
        f.write("\"")
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 将所有要加入训练的数据合并（readme + 编程语言、仓库名称、description）
info_csv = pd.read_csv('info_data.csv')
info_csv_len = len(info_csv)
print("description 仓库大小为：",info_csv_len)
with open('observe/readme_chinese.csv','r',encoding='utf-8') as f: # 
    readme_csv = f.readlines()
print("文件加载完毕")
readme_list = readme_csv[1:]
readme_csv_len = len(readme_list)
print("readme 仓库大小为：",readme_csv_len)
csv_filename = 'd_r_data.csv' #
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write("language,name,description,readme\n")
    for i in range(info_csv_len): #
        f.write(str(info_csv['language'][i]))
        f.write(",")
        f.write(str(info_csv['name'][i]))
        f.write(",")
        f.write(str(info_csv['description'][i]))
        f.write(",")
        f.write(readme_list[i])
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 只保留英文和几个符号
import re
with open('d_r_data.csv','r',encoding='utf-8') as f: # 
    readme_csv = f.readlines()
print("文件加载完毕")
readme_list = readme_csv[1:]
readme_csv_len = len(readme_list)
print("仓库大小为：",readme_csv_len)
#REPLACE_BY_SPACE_RE = re.compile('[/(){}[]|@,;.<>_《》]')
english_list = []
BAD_SYMBOLS_RE = re.compile('[^A-Z0-9a-z +.\-\\\\\\\']') # “0-9a-z”和“-.\'+_”会被保留下来
for i in range(readme_csv_len):
    text =  readme_list[i]
    text = BAD_SYMBOLS_RE.sub(' ',text)
    print(text)
    english_list.append(text)
#text = REPLACE_BY_SPACE_RE.sub(' ',text)
print(len(english_list))
csv_filename = 'english_only.csv' #
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('language,name,description,readme\n')
    for i in range(len(english_list)): #
        f.write(english_list[i])
        f.write("\n")
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 加载停用词表格
with open('rules/stopwords.txt', 'r') as f:
    stopwords_list = f.readlines()
stopwords_list = [i.strip() for i in stopwords_list]
STOPWORDS = set(stopwords_list)
print('停用词数量：', len(stopwords_list))
print('停用词列表：',STOPWORDS)

In [ ]:
# 去除停用词
with open('english_only.csv','r',encoding='utf-8') as f: # 
    readme_csv = f.readlines()
print("文件加载完毕")
readme_list = readme_csv[1:]
readme_csv_len = len(readme_list)
print("仓库大小为：",readme_csv_len)
readme_k = []
k_count = 0
for i in range(readme_csv_len):
    text = readme_list[i]
    text = text.lower()
    s_text = ' '.join([w for w in text.split() if w not in STOPWORDS]) # 删除停用词
    readme_k.append(s_text)
print(len(readme_k))
csv_filename = 'english_only3.csv' #
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write('language,name,description,readme\n')
    for i in range(readme_csv_len): #
        f.write(readme_k[i])
        f.write('\n')
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 数据准备
with open('english_only3.csv','r',encoding='utf-8') as f: # 
    readme_csv = f.readlines()
print("文件加载完毕")
readme_list = readme_csv[1:]
readme_csv_len = len(readme_list)
print("仓库大小为：",readme_csv_len)

In [ ]:
# 将文本文档转换为计数的稀疏矩阵
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_df=0.75,min_df=25,ngram_range=(1,2),token_pattern= '(\S+)') #
count_feature = cv.fit_transform(readme_list)
print(count_feature.shape)
print()
print(count_feature)
csv_filename = 'cv.text' #
with open(csv_filename, 'w', encoding='utf-8') as f:
    f.write(str(cv.vocabulary_))
print(f"Data saved to {csv_filename} successfully.")

In [ ]:
# 重点主题不在特征词频则扩充矩阵
import pandas as pd
topic_list = pd.read_csv('main256_topic.csv')
topics = list(topic_list['topics'])
topic_order = []
without_t = []
for i in range(256):
    if topics[i] not in cv.vocabulary_.keys():
        print(i)
        without_t.append(i)
        topic_order.append(0)
        continue
    topic_order.append(cv.vocabulary_[topics[i]])
for i in without_t:
    print(topics[i])
print(topic_order)

In [ ]:
print(topic_order[235])
topic_order[235] = count_feature.shape[1]
print(topic_order[235])
feature_arr = count_feature.toarray()
print(feature_arr.shape)
lst1 = np.array([[0]] * 12281)
print(lst1.shape)
count_feature = np.append(feature_arr,lst1,axis=1)
print(count_feature.shape)

In [ ]:
# 加上主题关联度特征
txt_filename = 'feature_vector/feature_without.txt'
with open(txt_filename,'r',encoding='utf-8') as f:
    feature = eval(f.read())
print(feature[1])

In [ ]:
print(count_feature.shape[0])
feature = np.array(feature, dtype=np.float64)
fea_add = []
percent = 0.25 # 定义关联度信息占比
for i in range(count_feature.shape[0]):
    one_f = count_feature[i]
    max_count = max(one_f)
    max_f = max(feature[i])
    mul_num = max_count*percent/max_f
    fea_add.append(mul_num * feature[i]) 

In [ ]:
print(count_feature[0][topic_order[0]])
print(topic_order[0])
print(fea_add[0])
print(len(fea_add[0]))

In [ ]:
# 按比例加入主题关联度特征
x_feature = []
count_feature_f = count_feature
for i in range(count_feature.shape[0]):
    new_feature = count_feature_f[i]
    for j in range(256):
        new_feature[topic_order[j]] += fea_add[i][j]
    x_feature.append(new_feature)

In [ ]:
# 计算词频矩阵的 tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

transform = TfidfTransformer()
tfidf_feature = transform.fit_transform(count_feature)
print(tfidf_feature.shape)
print()
print(tfidf_feature)

In [ ]:
import statistics
txt_filename = 'y.txt'
with open(txt_filename,'r',encoding='utf-8') as f:
    y = eval(f.read())
topic_num = []
for i in range(len(y)):
    topic_num.append(sum(y[i]))
mode = statistics.mode(topic_num)
median = statistics.mean(topic_num)
mean = statistics.mean(topic_num)
print("众数：",mode)
print("中位数：",median)
print("平均数：",mean)
y = np.array(y)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_feature, y, test_size=0.25, random_state = 50)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# 逻辑回归法
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics

clf_multilabel = OneVsRestClassifier(LogisticRegression(solver='lbfgs',max_iter=300,C=0.9))

clf_multilabel.fit(X_train,y_train) 
print("逻辑回归模型训练完毕！")

In [ ]:
val_pred = clf_multilabel.predict(X_test)  
val_pred_proba = clf_multilabel.predict_proba(X_test)
for i in range(len(y_test)):
    one_pred = list(val_pred[i])
    if sum(one_pred)==0:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-1])] = 1
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
    elif sum(one_pred)==1:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
 
p_score = metrics.precision_score(y_test,val_pred,average='micro')
print("该方法精确度为：",p_score)
r_score = metrics.recall_score(y_test,val_pred,average='micro') # 被模型正确预测为正类的样本占所有实际为正类的样本的比例
print("该方法召回率为：",r_score)  
F1_score = 2 * p_score * r_score / (p_score + r_score)
print("该方法 F1 分数为：",F1_score)
β = 0.8
Fβ_score = (1 + β * β) * p_score * r_score / (β * β * p_score + r_score)
print("该方法 Fβ 分数为：",Fβ_score)

In [ ]:
# 朴素贝叶斯法
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn import metrics

clf_multilabel = OneVsRestClassifier(MultinomialNB()) 

clf_multilabel.fit(X_train,y_train)  
print("朴素贝叶斯模型训练完毕！")

In [ ]:
val_pred = clf_multilabel.predict(X_test)
val_pred_proba = clf_multilabel.predict_proba(X_test)
for i in range(len(y_test)):
    one_pred = list(val_pred[i])
    if sum(one_pred)==0:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-1])] = 1
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
    elif sum(one_pred)==1:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
 
p_score = metrics.precision_score(y_test,val_pred,average='micro')
print("该方法精确度为：",p_score)
r_score = metrics.recall_score(y_test,val_pred,average='micro') 
print("该方法召回率为：",r_score)
F1_score = 2 * p_score * r_score / (p_score + r_score)
print("该方法 F1 分数为：",F1_score)
β = 0.8
Fβ_score = (1 + β * β) * p_score * r_score / (β * β * p_score + r_score)
print("该方法 Fβ 分数为：",Fβ_score)

In [ ]:
# 线性支持向量机法
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC 
from sklearn import metrics

clf_multilabel = OneVsRestClassifier(LinearSVC()) 

clf_multilabel.fit(X_train,y_train) 
print("线性支持向量机模型训练完毕！")

In [ ]:
val_pred = clf_multilabel.predict(X_test) 
val_pred_proba = clf_multilabel.decision_function(X_test)
for i in range(len(y_test)):
    one_pred = list(val_pred[i])
    if sum(one_pred)==0:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-1])] = 1
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
    elif sum(one_pred)==1:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
 
p_score = metrics.precision_score(y_test,val_pred,average='micro')
print("该方法精确度为：",p_score)
r_score = metrics.recall_score(y_test,val_pred,average='micro') 
print("该方法召回率为：",r_score) 
F1_score = 2 * p_score * r_score / (p_score + r_score)
print("该方法 F1 分数为：",F1_score)
β = 0.8
Fβ_score = (1 + β * β) * p_score * r_score / (β * β * p_score + r_score)
print("该方法 Fβ 分数为：",Fβ_score)

In [ ]:
# 决策树法
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

clf_multilabel = OneVsRestClassifier(DecisionTreeClassifier(max_depth=15)) 

clf_multilabel.fit(X_train,y_train)
print("决策树模型训练完毕！")

In [ ]:
val_pred = clf_multilabel.predict(X_test) 
val_pred_proba = clf_multilabel.predict_proba(X_test)
for i in range(len(y_test)):
    one_pred = list(val_pred[i])
    if sum(one_pred)==0:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-1])] = 1
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
    elif sum(one_pred)==1:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1 
 
p_score = metrics.precision_score(y_test,val_pred,average='micro')
print("该方法精确度为：",p_score)
r_score = metrics.recall_score(y_test,val_pred,average='micro') 
print("该方法召回率为：",r_score)  
F1_score = 2 * p_score * r_score / (p_score + r_score)
print("该方法 F1 分数为：",F1_score)
β = 0.8
Fβ_score = (1 + β * β) * p_score * r_score / (β * β * p_score + r_score)
print("该方法 Fβ 分数为：",Fβ_score)

In [ ]:
# K 近邻法
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

clf_multilabel = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=15)) 

clf_multilabel.fit(X_train,y_train) 
print("K 近邻模型训练完毕！")

In [ ]:
val_pred = clf_multilabel.predict(X_test)  
val_pred_proba = clf_multilabel.predict_proba(X_test)
for i in range(len(y_test)):
    one_pred = list(val_pred[i])
    if sum(one_pred)==0:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-1])] = 1
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
    elif sum(one_pred)==1:
        one_pred_proba = list(val_pred_proba[i])
        pred_proba = sorted(one_pred_proba)
        val_pred[i][one_pred_proba.index(pred_proba[-2])] = 1
 
p_score = metrics.precision_score(y_test,val_pred,average='micro')
print("该方法精确度为：",p_score)
r_score = metrics.recall_score(y_test,val_pred,average='micro') 
print("该方法召回率为：",r_score)  
F1_score = 2 * p_score * r_score / (p_score + r_score)
print("该方法 F1 分数为：",F1_score)
β = 0.8
Fβ_score = (1 + β * β) * p_score * r_score / (β * β * p_score + r_score)
print("该方法 Fβ 分数为：",Fβ_score)